![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [176]:
import pandas as pd
import numpy as np

file_csv = pd.read_csv('bank_marketing.csv')
file_csv.tail(n=20)

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
41168,41168,38,entrepreneur,married,university.degree,no,no,no,cellular,nov,5,144,2,999,0,nonexistent,-1.1,94.767,-50.8,1.030,4963.6,no
41169,41169,62,services,married,high.school,no,yes,no,cellular,nov,21,154,5,999,0,nonexistent,-1.1,94.767,-50.8,1.030,4963.6,no
41170,41170,40,management,divorced,university.degree,no,yes,no,cellular,nov,21,293,2,999,4,failure,-1.1,94.767,-50.8,1.030,4963.6,no
41171,41171,33,student,married,professional.course,no,yes,no,telephone,nov,10,112,1,999,0,nonexistent,-1.1,94.767,-50.8,1.031,4963.6,yes
41172,41172,31,admin.,single,university.degree,no,yes,no,cellular,nov,6,353,1,999,0,nonexistent,-1.1,94.767,-50.8,1.031,4963.6,yes
41173,41173,62,retired,married,university.degree,no,yes,no,cellular,nov,6,329,1,999,2,failure,-1.1,94.767,-50.8,1.031,4963.6,yes
41174,41174,62,retired,married,university.degree,no,yes,no,cellular,nov,14,208,1,1,6,success,-1.1,94.767,-50.8,1.031,4963.6,yes
41175,41175,34,student,single,unknown,no,yes,no,cellular,nov,24,180,1,999,2,failure,-1.1,94.767,-50.8,1.031,4963.6,no
41176,41176,38,housemaid,divorced,high.school,no,yes,yes,cellular,nov,6,360,1,999,0,nonexistent,-1.1,94.767,-50.8,1.031,4963.6,no
41177,41177,57,retired,married,professional.course,no,yes,no,cellular,nov,25,124,6,999,0,nonexistent,-1.1,94.767,-50.8,1.031,4963.6,no


### Spliting the Data
Spliting the data in 3 DataFrames: client_part, camping_part and economics_part.

In [177]:
client_part = file_csv[["client_id", "age", "job", "marital", "education", "credit_default","housing","loan"]]
client_part.head()

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [178]:
campaign_part = file_csv[["client_id","campaign","duration","pdays","previous","poutcome","y","day","month"]]
campaign_part.head()

,client_id,campaign,duration,pdays,previous,poutcome,y,day,month
0,0,1,261,999,0,nonexistent,no,13,may
1,1,1,149,999,0,nonexistent,no,19,may
2,2,1,226,999,0,nonexistent,no,23,may
3,3,1,151,999,0,nonexistent,no,27,may
4,4,1,307,999,0,nonexistent,no,3,may


In [ ]:
economics_part = file_csv[["client_id","emp_var_rate","cons_price_idx","euribor3m","nr_employed"]]
economics_part.head()

### Cleaning the data
Cleaning the split parts DataFrame

In [180]:
# Renaming the columns
client_part_clean = client_part.rename(columns={"client_id":"id"})
campaign_part_clean = campaign_part.rename(columns={"duration":"contact_duration","previous":"previous_campaing_contacts","y":"campaign_outcome","poutcome":"previous_outcome","campaign":"number_contacts"})
economics_part_clean = economics_part.rename(columns={"euribor3m":"euribor_three_months","nr_employed":"number_employed"})

# Printing DataFrames cleans
client_part_clean.head()
campaign_part_clean.head()
economics_part_clean.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [181]:
# Adding the colunm campaign_id
campaign_part_clean['month'] = campaign_part_clean['month'].str.capitalize()
campaign_part_clean['year'] = '2022'
campaign_part_clean['campaign_id'] = '1'
campaign_part_clean['day'] = campaign_part_clean['day'].astype(str)
campaign_part_clean['last_contact_date'] = campaign_part_clean['year']+'-'+campaign_part_clean['month']+'-'+campaign_part_clean['day']

In [182]:
# Cleaning the columns client
client_part_clean['education'] = client_part_clean['education'].str.replace('.','_').replace('unknown', np.NaN)
client_part_clean['job'] = client_part_clean['job'].str.replace('.',' ')

# Cleaning the columns campaign
campaign_part_clean['previous_outcome'] = campaign_part_clean['previous_outcome'].replace('nonexistent',np.NaN).replace('success',1).replace('failure',0)

campaign_part_clean['campaign_outcome'] = campaign_part_clean['campaign_outcome'].replace('success',1).replace('failure',0)

# Converting the date column
campaign_part_clean['last_contact_date'] = pd.to_datetime(campaign_part_clean['last_contact_date'], format='%Y-%b-%d')

campaign_part_clean.tail(n=20)

,client_id,number_contacts,contact_duration,pdays,previous_campaing_contacts,previous_outcome,campaign_outcome,day,month,year,campaign_id,last_contact_date
41168,41168,2,144,999,0,NaN,no,5,Nov,2022,1,2022-11-05
41169,41169,5,154,999,0,NaN,no,21,Nov,2022,1,2022-11-21
41170,41170,2,293,999,4,0.0,no,21,Nov,2022,1,2022-11-21
41171,41171,1,112,999,0,NaN,yes,10,Nov,2022,1,2022-11-10
41172,41172,1,353,999,0,NaN,yes,6,Nov,2022,1,2022-11-06
41173,41173,1,329,999,2,0.0,yes,6,Nov,2022,1,2022-11-06
41174,41174,1,208,1,6,1.0,yes,14,Nov,2022,1,2022-11-14
41175,41175,1,180,999,2,0.0,no,24,Nov,2022,1,2022-11-24
41176,41176,1,360,999,0,NaN,no,6,Nov,2022,1,2022-11-06
41177,41177,6,124,999,0,NaN,no,25,Nov,2022,1,2022-11-25


In [183]:
# removing the colunm campaign_id
campaign_part_clean.drop(columns='month',inplace=True)
campaign_part_clean.drop(columns='day',inplace=True)
campaign_part_clean.drop(columns='year',inplace=True)

In [184]:
campaign_part_clean.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaing_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,no,1,2022-05-13
1,1,1,149,999,0,NaN,no,1,2022-05-19
2,2,1,226,999,0,NaN,no,1,2022-05-23
3,3,1,151,999,0,NaN,no,1,2022-05-27
4,4,1,307,999,0,NaN,no,1,2022-05-03


### Saving the DataFrames in csv

In [185]:
client_part_clean.to_csv('client.csv',index=False)
campaign_part_clean.to_csv('campaign.csv',index=False)
economics_part_clean.to_csv('economics.csv',index=False)

In [186]:
client_part_clean.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


### Designing the database
Creating the tables and records


## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |


In [187]:
# Creating the client table
client_table = """
CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |


In [188]:
# Creating the campaign table
campaign_table = """
CREATE TABLE campaign 
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |


In [189]:
# Creating the economics table
economics_table = """
CREATE TABLE economics 
(
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
)
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

In [ ]:
var1 = input('digite var 1: ')
var2 = input('digite var 2: ')

if len(var1) == len(var2):
    print('{0},{1}tamanho das variaveis: {3}',var1,var2,len(var1) )
elif type(var1) == type(var2):
    print('As duas variaveis são do tipo string!')